In [1]:
from google.colab import drive

# Monta o Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install openpyxl

##GCP

In [3]:
from google.colab import auth
auth.authenticate_user()

Transferir Dados para o GCP

In [7]:
import pandas as pd
from pathlib import Path
import subprocess

# ---------- CONFIGURAÇÕES ----------
BUCKET = "brats-final-training-818"

excel_path = "/content/drive/MyDrive/TCC/test_subset_818.xlsx"

gcs_base = f"gs://{BUCKET}/Task001_MyBraTS"
imagesTr = f"{gcs_base}/imagesTr"
labelsTr = f"{gcs_base}/labelsTr"

data_folders = [
    Path("/content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-additionaltrainingdata/training_data_additional"),
    Path("/content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-trainingdata/training_data1_v2"),
    Path("/content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-validationdata/validation_data")
]

# ---------- FUNÇÃO DE VERIFICAÇÃO ----------
def gcs_exists(path):
    """Retorna True se o arquivo existir no bucket."""
    result = subprocess.run(
        ["gsutil", "-q", "stat", path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.returncode == 0


# ---------- FUNÇÃO PARA CHECAR SE PACIENTE JÁ ESTÁ COMPLETO NO BUCKET ----------
def patient_complete_in_bucket(pid):
    """Verifica se TODOS os arquivos esperados do paciente estão no bucket."""
    modalities = ["t1n", "t1c", "t2f", "t2w"]
    expected_files = [f"{pid}-{100}-{m}.nii.gz" for m in modalities]
    expected_seg   = f"{pid}-{100}-seg.nii.gz"

    all_exist = True

    # Check images
    for fname in expected_files:
        if not gcs_exists(f"{imagesTr}/{fname}"):
            all_exist = False

    # Check seg
    if not gcs_exists(f"{labelsTr}/{expected_seg}"):
        all_exist = False

    return all_exist


# ---------- LEITURA DO EXCEL ----------
df = pd.read_excel(excel_path)
patient_ids = df['PatientID'].unique()

# ---------- LOOP PRINCIPAL ----------
for pid in patient_ids:
    print(f"\n🔎 Checando paciente: {pid}")

    # 1) Verificação antes de procurar no Drive
    if patient_complete_in_bucket(pid):
        print(f"  ✅ Todos arquivos já existem no bucket → pulando paciente.")
        continue

    print(f"  ❌ Paciente incompleto no bucket → procurando no Drive...")

    # 2) Procurar no Drive porque algo faltou no bucket
    found = False

    for data_folder in data_folders:
        for visit_folder in data_folder.rglob("*"):
            if visit_folder.is_dir() and pid in visit_folder.name:
                found = True
                print(f"  📁 Encontrado no Drive: {visit_folder}")

                # Copiar imagens
                for src_file in visit_folder.glob("*.nii*"):
                    if "seg" in src_file.name.lower():
                        continue
                    dst_path = f"{imagesTr}/{src_file.name}"

                    if gcs_exists(dst_path):
                        print(f"    ⚠ Já existe: {src_file.name}")
                    else:
                        !gsutil cp "{src_file}" "{dst_path}"
                        print(f"    ⬆ Upload: {src_file.name}")

                # Copiar segmentações
                for seg_file in visit_folder.glob("*seg*.nii*"):
                    dst_path = f"{labelsTr}/{seg_file.name}"

                    if gcs_exists(dst_path):
                        print(f"    ⚠ Segmentação já existe: {seg_file.name}")
                    else:
                        !gsutil cp "{seg_file}" "{dst_path}"
                        print(f"    ⬆ Upload segmentação: {seg_file.name}")

                break

        if found:
            break

    if not found:
        print(f"  ❗ Paciente {pid} não encontrado no Drive.")

print("\n✅ Upload concluído! Verificação inteligente ativada (bucket → drive).")

A saída de streaming foi truncada nas últimas 5000 linhas.
  ❌ Paciente incompleto no bucket → procurando no Drive...
  📁 Encontrado no Drive: /content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-trainingdata/training_data1_v2/BraTS-GLI-02246-100
Copying file:///content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-trainingdata/training_data1_v2/BraTS-GLI-02246-100/BraTS-GLI-02246-100-t1c.nii.gz [Content-Type=application/octet-stream]...
- [1 files][  5.2 MiB/  5.2 MiB]                                                
Operation completed over 1 objects/5.2 MiB.                                      
    ⬆ Upload: BraTS-GLI-02246-100-t1c.nii.gz
Copying file:///content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-trainingdata/training_data1_v2/BraTS-GLI-02246-100/BraTS-GLI-02246-100-t2w.nii.gz [Content-Type=application/octet-stream]...
- [1 files][  5.2 MiB/  5.2 MiB]                                                
Operation completed over 1 objects/5.2 MiB.  

Renomear arquivos

In [ ]:
import subprocess
import re

BUCKET = "brats-final-training-818"
gcs_imagesTr = f"gs://{BUCKET}/Task001_MyBraTS/imagesTr"

# Mapeamento das modalidades → sufixo nnU-Net
modality_map = {
    "-t1n": "_0000",
    "-t1c": "_0001",
    "-t2w": "_0002",
    "-t2f": "_0003"
}

def gcs_ls(prefix):
    """Lista arquivos no GCS."""
    result = subprocess.run(
        ["gsutil", "ls", prefix],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    return result.stdout.strip().split("\n")

def gcs_exists(path):
    """Verifica se o arquivo existe no GCS."""
    result = subprocess.run(
        ["gsutil", "-q", "stat", path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    return result.returncode == 0


# ------------------------------------------------------------------------------------
#                               LOOP DE RENOMEAÇÃO
# ------------------------------------------------------------------------------------

files = gcs_ls(f"{gcs_imagesTr}/*.nii*")

print(f"Encontrados {len(files)} arquivos no bucket.\n")

for full_path in files:
    filename = full_path.split("/")[-1]
    stem = re.sub(r"\.nii(\.gz)?$", "", filename)   # remove .nii ou .nii.gz

    new_stem = stem
    found = False

    for mod_key, mod_id in modality_map.items():
        pattern = re.compile(mod_key, re.IGNORECASE)
        if re.search(pattern, stem):
            new_stem = re.sub(pattern, mod_id, stem)
            found = True
            break

    if not found:
        print(f"⚠️ Modalidade não encontrada em: {filename}")
        continue

    ext = filename[len(stem):]   # mantém ".nii" ou ".nii.gz"
    new_filename = f"{new_stem}{ext}"
    new_path = f"{gcs_imagesTr}/{new_filename}"

    # Verificar se já existe
    if gcs_exists(new_path):
        print(f"⚠️ Já existe no bucket, pulando: {new_filename}")
        continue

    # Renomeia: cp → rm
    print(f"Renomeando {filename} → {new_filename}")
    subprocess.run(["gsutil", "cp", full_path, new_path])
    subprocess.run(["gsutil", "rm", full_path])

print("\n✅ Renomeação concluída com sucesso!")


Encontrados 3272 arquivos no bucket.

Renomeando BraTS-GLI-00005-100-t1c.nii.gz → BraTS-GLI-00005-100_0001.nii.gz
Renomeando BraTS-GLI-00005-100-t1n.nii.gz → BraTS-GLI-00005-100_0000.nii.gz
Renomeando BraTS-GLI-00005-100-t2f.nii.gz → BraTS-GLI-00005-100_0003.nii.gz
Renomeando BraTS-GLI-00005-100-t2w.nii.gz → BraTS-GLI-00005-100_0002.nii.gz
Renomeando BraTS-GLI-00006-100-t1c.nii.gz → BraTS-GLI-00006-100_0001.nii.gz
Renomeando BraTS-GLI-00006-100-t1n.nii.gz → BraTS-GLI-00006-100_0000.nii.gz
Renomeando BraTS-GLI-00006-100-t2f.nii.gz → BraTS-GLI-00006-100_0003.nii.gz
Renomeando BraTS-GLI-00006-100-t2w.nii.gz → BraTS-GLI-00006-100_0002.nii.gz
Renomeando BraTS-GLI-00008-100-t1c.nii.gz → BraTS-GLI-00008-100_0001.nii.gz
Renomeando BraTS-GLI-00008-100-t1n.nii.gz → BraTS-GLI-00008-100_0000.nii.gz
Renomeando BraTS-GLI-00008-100-t2f.nii.gz → BraTS-GLI-00008-100_0003.nii.gz
Renomeando BraTS-GLI-00008-100-t2w.nii.gz → BraTS-GLI-00008-100_0002.nii.gz
Renomeando BraTS-GLI-00009-100-t1c.nii.gz → BraTS-

##Drive

In [4]:
import pandas as pd
from pathlib import Path
import shutil

# ---------- CONFIGURAÇÕES ----------

excel_path = "/content/drive/MyDrive/TCC/test_subset_818.xlsx"
raw_data_base = Path("/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_818/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS")

imagesTr = raw_data_base / "imagesTr"
labelsTr = raw_data_base / "labelsTr"

data_folders = [
    Path("/content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-additionaltrainingdata/training_data_additional"),
    Path("/content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-trainingdata/training_data1_v2"),
    Path("/content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-validationdata/validation_data")
]

# Cria pastas de destino
imagesTr.mkdir(parents=True, exist_ok=True)
labelsTr.mkdir(parents=True, exist_ok=True)

# ---------- LEITURA DO EXCEL ----------

df = pd.read_excel(excel_path)
patient_ids = df['PatientID'].unique()

# ---------- LOOP PRINCIPAL ----------
for pid in patient_ids:
    print(f"\nProcurando paciente: {pid}")
    found = False

    for data_folder in data_folders:
        for visit_folder in data_folder.rglob("*"):
            if visit_folder.is_dir() and pid in visit_folder.name:
                visit_id = visit_folder.name
                found = True
                print(f"  ✅ Primeira aparição encontrada: {visit_folder}")

                # Copia todas as imagens multimodais (mantendo nomes originais)
                for src_file in visit_folder.glob("*.nii*"):
                    # Pula arquivos de segmentação
                    if "seg" in src_file.name.lower():
                        continue
                    dst_file = imagesTr / src_file.name
                    shutil.copy(src_file, dst_file)
                    print(f"    Copiado: {src_file.name}")

                # Copia apenas segmentações
                for seg_file in visit_folder.glob("*seg*.nii*"):
                    dst_file = labelsTr / seg_file.name
                    shutil.copy(seg_file, dst_file)
                    print(f"    Segmentação copiada: {seg_file.name}")

                # Sai do loop após a primeira aparição
                break

        if found:
            break  # Sai do loop de data_folder também

    if not found:
        print(f"Paciente {pid} não encontrado em nenhuma pasta.")

print("\n✅ Dataset convertido para o padrão nnU-Net (mantendo nomes originais e segmentações separadas).")


Procurando paciente: BraTS-GLI-02754
  ✅ Primeira aparição encontrada: /content/drive/MyDrive/TCC/1.Data/BraTS-GLI/brats2024-brats-gli-trainingdata/training_data1_v2/BraTS-GLI-02754-100
    Copiado: BraTS-GLI-02754-100-t2f.nii.gz
    Copiado: BraTS-GLI-02754-100-t2w.nii.gz
    Copiado: BraTS-GLI-02754-100-t1c.nii.gz
    Copiado: BraTS-GLI-02754-100-t1n.nii.gz
    Segmentação copiada: BraTS-GLI-02754-100-seg.nii.gz

Procurando paciente: BraTS-GLI-02272


KeyboardInterrupt: 

In [ ]:
from pathlib import Path
import re

# ---------- CONFIGURAÇÃO ----------
imagesTr = Path("/content/drive/MyDrive/TCC/1.Data/nnU-net_format/subset_818/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS/imagesTr")

# Mapeamento das modalidades para sufixo nnU-Net
modality_map = {
    "-t1n": "_0000",
    "-t1c": "_0001",
    "-t2w": "_0002",
    "-t2f": "_0003"
}

# ---------- RENOMEAR ARQUIVOS ----------
for img_file in imagesTr.glob("*.nii*"):
    original_name = img_file.stem  # sem extensão

    new_name_part = original_name
    found = False

    for mod_key, mod_id in modality_map.items():
        # Procura a modalidade ignorando case
        pattern = re.compile(mod_key, re.IGNORECASE)
        if re.search(pattern, original_name):
            new_name_part = re.sub(pattern, mod_id, original_name)
            found = True
            break

    if found:
        new_path = imagesTr / f"{new_name_part}{img_file.suffix}"
        img_file.rename(new_path)
        print(f"Renomeado: {img_file.name} -> {new_path.name}")
    else:
        print(f"⚠️ Modalidade não encontrada em: {img_file.name}")

Renomeado: BraTS-GLI-02636-100-t1c.nii.gz -> BraTS-GLI-02636-100_0001.nii.gz
Renomeado: BraTS-GLI-02636-100-t1n.nii.gz -> BraTS-GLI-02636-100_0000.nii.gz
Renomeado: BraTS-GLI-02636-100-t2f.nii.gz -> BraTS-GLI-02636-100_0003.nii.gz
Renomeado: BraTS-GLI-02636-100-t2w.nii.gz -> BraTS-GLI-02636-100_0002.nii.gz
Renomeado: BraTS-GLI-02208-100-t1c.nii.gz -> BraTS-GLI-02208-100_0001.nii.gz
Renomeado: BraTS-GLI-02208-100-t2w.nii.gz -> BraTS-GLI-02208-100_0002.nii.gz
Renomeado: BraTS-GLI-02208-100-t2f.nii.gz -> BraTS-GLI-02208-100_0003.nii.gz
Renomeado: BraTS-GLI-02208-100-t1n.nii.gz -> BraTS-GLI-02208-100_0000.nii.gz
Renomeado: BraTS-GLI-02336-100-t1c.nii.gz -> BraTS-GLI-02336-100_0001.nii.gz
Renomeado: BraTS-GLI-02336-100-t1n.nii.gz -> BraTS-GLI-02336-100_0000.nii.gz
Renomeado: BraTS-GLI-02336-100-t2f.nii.gz -> BraTS-GLI-02336-100_0003.nii.gz
Renomeado: BraTS-GLI-02336-100-t2w.nii.gz -> BraTS-GLI-02336-100_0002.nii.gz
Renomeado: BraTS-GLI-02421-100-t1c.nii.gz -> BraTS-GLI-02421-100_0001.nii.gz

In [ ]:
from pathlib import Path
import re

# ---------- CONFIGURAÇÃO ----------
base_dir = Path("/content/drive/MyDrive/TCC_1/1.Data/nnU-net_format/subset_50_PCA/nnUNet_raw_data_base/nnUNet_raw_data/Task001_MyBraTS")
labelsTs = base_dir / "labelsTs"
imagesTs = base_dir / "imagesTs"
labelsTr = base_dir / "labelsTr"

# ---------- FUNÇÃO PARA RENOMEAR ----------
def remove_seg_suffix(folder: Path):
    for file in folder.glob("*.nii*"):
        original_name = file.name
        new_name = re.sub(r"-seg", "", original_name, flags=re.IGNORECASE)
        if new_name != original_name:
            new_path = folder / new_name
            file.rename(new_path)
            print(f"Renomeado: {original_name} → {new_name}")
        else:
            print(f"Sem alteração: {original_name}")

# ---------- EXECUÇÃO ----------
print("🧠 Renomeando labelsTs...")
remove_seg_suffix(labelsTs)

print("\n🩻 Renomeando labelsTr...")
remove_seg_suffix(labelsTr)

print("\n✅ Renomeação concluída!")


🧠 Renomeando labelsTs...
Sem alteração: BraTS-GLI-02336-100.nii.gz
Sem alteração: BraTS-GLI-02867-100.nii.gz
Sem alteração: BraTS-GLI-02153-100.nii.gz
Sem alteração: BraTS-GLI-02685-100.nii.gz
Sem alteração: BraTS-GLI-02144-100.nii.gz
Sem alteração: BraTS-GLI-02152-100.nii.gz
Sem alteração: BraTS-GLI-00485-100.nii.gz
Sem alteração: BraTS-GLI-02244-100.nii.gz
Sem alteração: BraTS-GLI-02115-100.nii.gz
Sem alteração: BraTS-GLI-02228-100.nii.gz

🩻 Renomeando labelsTr...
Renomeado: BraTS-GLI-02636-100-seg.nii.gz → BraTS-GLI-02636-100.nii.gz
Renomeado: BraTS-GLI-02208-100-seg.nii.gz → BraTS-GLI-02208-100.nii.gz
Renomeado: BraTS-GLI-02421-100-seg.nii.gz → BraTS-GLI-02421-100.nii.gz
Renomeado: BraTS-GLI-02339-100-seg.nii.gz → BraTS-GLI-02339-100.nii.gz
Renomeado: BraTS-GLI-02487-100-seg.nii.gz → BraTS-GLI-02487-100.nii.gz
Renomeado: BraTS-GLI-02375-100-seg.nii.gz → BraTS-GLI-02375-100.nii.gz
Renomeado: BraTS-GLI-02941-100-seg.nii.gz → BraTS-GLI-02941-100.nii.gz
Renomeado: BraTS-GLI-02615-100-s